La base essentielle a connaitre : 

# Reindexation

UNE METHODE IMPORTANTE POUR LES OBJET PANDAS EST REINDEX, qui consiste à créer un nouvel objet dont les données se conforment à un nouvel index. Prenons exemple:

In [39]:
import pandas as pd

obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

l'appel à reindex sur cette série réorganise les données en fonction du nouvel index, en introduisant des valeurs manquantes si aucune valeur d'index n'etait déja présente:

In [40]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

Pour les données ordonnées, comme les séries chronologique, il peut etre souhaitable de faire une certaine interpolation ou de remplir des valeur lors de la réindexation, l'option METHODE nous permet de le faire, en spécifiant une méthode telle que ffill, qui préremplit les valeur :

In [41]:
obj3 = pd.Series(['bleu', 'pourpre', 'jaune'], index=[0, 2, 4])
obj3

0       bleu
2    pourpre
4      jaune
dtype: object

In [42]:
obj3.reindex(range(6), method='ffill')

0       bleu
1       bleu
2    pourpre
3    pourpre
4      jaune
5      jaune
dtype: object

Avec un dataframe, reindex peut modifier soit l'index (ligne), soit les colonnes, ou les deux. Si seule une séquence est passé, elle reindexe les lignes du résultat : 

In [43]:
import numpy as np
frame = pd.DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'], columns=['Ohio', 'Texas', 'Californie'])
frame

,Ohio,Texas,Californie
a,0,1,2
c,3,4,5
d,6,7,8


In [44]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,Californie
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


Les colonnes peuvent être reindexées avec le mot clé columns :

In [45]:
etats = ['Texas', 'Californie']
frame.reindex(columns=etats)

,Texas,Californie
a,1,2
c,4,5
d,7,8


Comme on le reverra plus en détail, on peut réindexer de maniere plus succincte en indexant les étiquettes avec loc, et de nombreux utilisateurs préfèrent n'utiliser que cette façon de procéder:

In [46]:
frame.loc[['a', 'c', 'd'], etats]

,Texas,Californie
a,1,2
c,4,5
d,7,8


#### Arguments de la fonction Index

index = nouvelle séquence à utiliser comme index. Peut être une instance d'Index ou toute autre structure de données Python semblable a une séquence. Un index sera utilisé exactement tel quel sans aucune copie.

méthode = Méthode d'interpolation (remplissage); 'ffill' remplit vers l'avant, tandis que 'bfill' remplit vers l'arriere

fill_value = Valeur de substitution à utiliser lors de l'introduction de données manquantes par réindexation.

limit = lors d'un remplissage vers l'avant ou vers l'arriere, taille maximal des trous (en nombre d'éléments) à combler.

tolérance = Lors d'un remplissage vers l'avant ou vers l'arriere, taille maximal des trous (en distance numérique absolue) à combler en cas de concordance inexacte.

level = Diffusion à travers un niveau, correspondant aux valeurs de l'indice sur le niveau MultiIndex passé

copy = Si True, copie toujours les données sous jacente même si le nouvel Index est équivalent a l'ancien; 
       Si False, ne copie pas les données lorsque que les index sont équivalents

#

# Supprimer des entrées d'un axe
il est facile de supprimer un ou plusieur entrées d'un axe si vous disposez déja d'un tableau d'index ou d'une liste sans ces entrées.Cela peut nécessiter un peu de manipulation de données et de logique ensembliste; la méthode drop peut renvoyer un nouvel objet avec la ou les valeurs indiquées supprimées d'un axe :

In [47]:
obj = pd.Series(np.arange(5.), index=['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [48]:
nouv_obj = obj.drop("c")
nouv_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

In [49]:
obj.drop(['d', 'c'])

a    0.0
b    1.0
e    4.0
dtype: float64

Avec un DataFrame, les valeur de l'index peuvent être supprimées de l'un ou de l'autre axe. Pour illustrer ceci, commençons par créer un exemple de DataFrame

In [50]:
data = pd.DataFrame(np.arange(16).reshape((4, 4)), index=['Ohio', 'Colorado', 'Utah', 'New York'], columns=['un','deux','trois','quatre'])
data

,un,deux,trois,quatre
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


L'appel à drop avec une séquence d'étiquettes supprimera les valeurs depuis les étiquettes des lignes (axe0) :

In [51]:
data.drop(['Colorado','Ohio'])

,un,deux,trois,quatre
Utah,8,9,10,11
New York,12,13,14,15


On peut supprimer des valeurs à partir de la colonnes en passant l'un des arguments axis=1 ou axis='columns':

In [52]:
data.drop('deux', axis=1)

,un,trois,quatre
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [53]:
data.drop(['trois', 'quatre'], axis='columns')

,un,deux
Ohio,0,1
Colorado,4,5
Utah,8,9
New York,12,13


de nombeux fonctions, comme drop, qui modifient la taille ou la forme d'un élément Series ou Dataframe, peuvent manipuler un objet en place sans retourner un nouvel objet:

In [54]:
obj.drop('c', inplace=True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

Faire attention à l'emploie de inplace, car il détruit toute les données qui sont supprimées

# Indexation, Selection et filtrage

l'indexation Series (obj[...]) fonctionne de maniere analogue à l'indexation des tableaux Numpy, sauf que vous pouvez utiliser les valeurs d'index des objets Series au lieu des seuls entiers. En voici quelque exemples :

In [55]:
obj  = pd.Series(np.arange(4.), index=['a','b','c','d'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [56]:
obj['b']

np.float64(1.0)

In [58]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [60]:
obj[['b','a','d']]

b    1.0
a    0.0
d    3.0
dtype: float64

In [62]:
obj[obj < 2]

a    0.0
b    1.0
dtype: float64

Le tranchage avec étiquette se comporte différemment du tranchage normal de python, en ce sens que la borne de fin est inclusive, comme dans :

In [63]:
obj['b': 'c']

b    1.0
c    2.0
dtype: float64

Effectuer une affectation à l'aide de ces méthode modifie la section correspondante de l'objet Series :

In [65]:
obj['b':'c'] = 5
obj

a    0.0
b    5.0
c    5.0
d    3.0
dtype: float64

l'indexation dans un DataFrame permet de récupérer une ou plusieur colonnes avec une seule valeur ou séquence :

In [75]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index=['Ohio','Colorado','Utha','New York'], 
                    columns=['un','deux','trois','quatre'])
data

,un,deux,trois,quatre
Ohio,0,1,2,3
Colorado,4,5,6,7
Utha,8,9,10,11
New York,12,13,14,15


In [76]:
data['deux']

Ohio         1
Colorado     5
Utha         9
New York    13
Name: deux, dtype: int64

In [77]:
data[['trois', 'un']]

,trois,un
Ohio,2,0
Colorado,6,4
Utha,10,8
New York,14,12


Une indexation de ce type comporte quelque cas particulier: tout d'abord, le tranchage ou selection des données à l'aide d'un tableau booleen :

In [78]:
data[:2]

,un,deux,trois,quatre
Ohio,0,1,2,3
Colorado,4,5,6,7


In [79]:
data[data['trois'] > 5]

,un,deux,trois,quatre
Colorado,4,5,6,7
Utha,8,9,10,11
New York,12,13,14,15


La synthaxe de selection des ligne [:2] est fournit a titre commodité. le fait de passé par un élement unique ou une liste à l'opérateur [] sélectionne des colonnes.
Un autre cas d'utilisation de l'indexation avec DataFrame Boolén, comme celui produit par une comparaison scalaire :

In [80]:
data < 5

,un,deux,trois,quatre
Ohio,True,True,True,True
Colorado,True,False,False,False
Utha,False,False,False,False
New York,False,False,False,False


In [82]:
data[data < 5] = 0
data

,un,deux,trois,quatre
Ohio,0,0,0,0
Colorado,0,5,6,7
Utha,8,9,10,11
New York,12,13,14,15


Dans ce cas particulier, la syntaxe de DataFrame ressemble davantage à celle d'un tableau Numpy bidimensionnel.

# Sélection avec loc et iloc
pour l'indexation par étiquettes des DataFrame sur les lignes, les opérateurs d'indexation spéciaux loc et iloc permettent de sélectionner un sous-ensembles de lignes et de colonnes d'un DataFrame avce une notation de type Numpy en utilisant soit des étiquettes d'axes(loc), soit d'entier(iloc)

Exemple on va selectioner une seule ligne et plusieur colonnes par étiquette :

In [84]:
data.loc['Colorado',['deux','trois']]

deux     5
trois    6
Name: Colorado, dtype: int64

Effectuons ensuite quelques sélection similaire avec des entiers en utilisant iloc:

In [87]:
data.iloc[2,[3,0,1]]

quatre    11
un         8
deux       9
Name: Utha, dtype: int64

In [89]:
data #data principal

,un,deux,trois,quatre
Ohio,0,0,0,0
Colorado,0,5,6,7
Utha,8,9,10,11
New York,12,13,14,15


In [90]:
data.iloc[2]

un         8
deux       9
trois     10
quatre    11
Name: Utha, dtype: int64

In [91]:
data.iloc[[1,2], [3,0,1]]

,quatre,un,deux
Colorado,7,0,5
Utha,11,8,9


Les deux fonction d'indexation travaillent avec des tranches en plus d'étiquettes individuelle ou liste d'étiquettes :

In [92]:
data.loc[:'Utha','deux']

Ohio        0
Colorado    5
Utha        9
Name: deux, dtype: int64

In [93]:
data.iloc[:, :3][data.trois > 5]

,un,deux,trois
Colorado,0,5,6
Utha,8,9,10
New York,12,13,14


Il existe donc de nombreuses façon de sélectionner et de réorganisé les données contenues dans un objet pandas.

##### Option d'indexation avec les DataFrame : 

df[val] = Selectionne une seule colone ou une séquence de colonne dans la DataFrame: cas particulier : tableau booléen(filtre des lignes), tranche(tranches des lignes) ou dataFrame booleen(définit des valeur en fonction de certaind critères).

df.loc[val] = Selectionne par étiquette une seule ligne ou un sous ensemble de ligne dans la dataFrame

df.loc[:, val] = Selectionne par étiquette une seule colonne ou un sous ensemble de colonnes

df.loc[val1, val2] = Selectionne par étiquette à la fois des lignes et des colonnes.

df.iloc[pos] = Selectionne par position entiere une seule ligne ou un sous ensemble de ligne  dans la DataFrame.

df.iloc[:, pos] = Selectionne par position entière une seule colonne ou un sous ensemble de colonne

df.iloc[pos_i, pos_j] = Selectionne a la fois des lignes et des colonnes par position entiere

df.at[etiq_i, etiq_j] = Selectionne une seule valeur scalaire par étiquette de lignes et de colonne

df.iat[i, j] = Selectionne une seule valeur scalaire par position de ligne et de colonne(nombre entiers)

méthode.réindex = Slectionne des lignes ou des colonnes par étiquettes.

méthode get_value, set_value = Selectionne une valeur unique par étiquette de ligne et de colonne.






